***GENERATED CODE FOR aggregationrecommender PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

In [ ]:
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class HDFSConnector:

    def fetch(spark, config):
        df = spark.read.\
            options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                    inferschema='true',
                    delimiter=eval(config)["delimiter"])\
            .csv("hdfs://34.83.203.43:54310"+eval(config)['url'])
        display(df.limit(2).toPandas())
        return df

    def put(df, spark, config):
        return df.write.format('csv').options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                                              delimiter=eval(config)["delimiter"]).save(("%s %s") % (datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")+"_", eval(config)['url']))


***OPERATION FUNCTIONS***

***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
import json
from pyspark.sql.functions import col, when
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def replaceByMean(self, feature, df, mean_=-1):

        meanValue = df.select(mean(col(feature.name)).alias(
            'mean')).collect()[0]["mean"]
        df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                                         meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        maxValue = df.select(max(col(feature.name)).alias('max')).collect()[
            0]["max"]
        df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        minValue = df.select(min(col(feature.name)).alias('min')).collect()[
            0]["min"]
        df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        stddevValue = df.select(stddev(col(feature.name)).alias(
            'stddev')).collect()[0]["stddev"]
        df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        fillValue = df.where(col(feature.name).isNotNull()
                             ).head(1)[0][feature.name]
        df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        display(transformationDF.limit(2).toPandas())
        return transformationDF


***AUTOML FUNCTIONS***

In [ ]:
import json
import pandas as pd
from scipy.sparse import csr_matrix
from lightfm import LightFM
import numpy as np


def sparseMatrix(df, user, product, rating):
    df_selected = df[[user, product, rating]]
    user_df = pd.pivot_table(df_selected, index=user,
                             columns=product, values=rating)
    # fill missing values with 0
    user_df = user_df.fillna(0)
    user_id = list(user_df.index)
    user_dict = {}
    counter = 0
    for i in user_id:
        user_dict[i] = counter
        counter += 1
    # convert to csr matrix
    user_df_csr = csr_matrix(user_df.values)
    return user_df, user_df_csr, user_id, user_dict


def recommenderModel(user_df_csr):
    model = LightFM()
    model = model.fit(user_df_csr)
    return model


def recommendPredict(model, interactions, user_id, user_dict, threshold=0, nrec_items=10, show=True):
    n_users, n_items = interactions.shape
    user_x = user_dict[user_id]
    scores = pd.Series(model.predict(user_x, np.arange(n_items)))
    scores.index = interactions.columns
    scores = list(pd.Series(scores.sort_values(ascending=False).index))

    known_items = list(pd.Series(interactions.loc[user_id, :]
                                 [interactions.loc[user_id, :] > threshold].index).sort_values(ascending=False))

    scores = [x for x in scores if x not in known_items]
    return_score_list = scores[0:nrec_items]
    known_items = list(pd.Series(known_items))
    scores = list(pd.Series(return_score_list))
    known_likes = {}
    recommend_dict = []
    counter = 1
    for i in known_items:
        known_likes[counter] = i
        counter += 1
    counter = 1
    for i in scores:
        recommend_dict.append(i)
        counter += 1

    if show == True:
        print("User: " + str(user_id))
        print("Known Likes:")
        counter = 1
        for i in known_items:
            print(str(counter) + '- ' + str(i))
            counter += 1
        print("\n Recommended Items:")
        counter = 1
        for i in scores:
            print(str(counter) + '- ' + str(i))
            counter += 1
    return known_likes, recommend_dict


def recommend(df, originalfile, user, product, rating):
    df = df.toPandas()
    total_user = df[user].to_list()
    user_df, user_df_csr, user_id, user_dict = sparseMatrix(
        df, user, product, rating)
    model = recommenderModel(user_df_csr)
    recommend_dict_all = []
    for us in total_user:
        recommend_obj = {}
        known_likes, recommend_dict = recommendPredict(
            model, user_df, us, user_dict, threshold=0, nrec_items=20, show=False)
        recommend_obj["recommendedOn"] = us
        recommend_obj["recomendations"] = recommend_dict
        recommend_dict_all.append(recommend_obj)
        display(recommend_dict_all)
    return recommend_dict_all


class Recommender:

    def run(spark_DF, spark, config):
        stageAttributes = json.loads(config)
        stageAttributes['model']
        stageAttributes['user']
        stageAttributes['product']
        stageAttributes['rating']
        recommender_obj = recommend(df=spark_DF)
        [recommender_obj]


***READING DATAFRAME***

In [ ]:
#%run aggregationrecommenderHooks.ipynb
try:
	#sourcePreExecutionHook()

	hotelrecommendationstrain = HDFSConnector.fetch(spark, "{'url': '/FileStore/platform/demo/Hotel_Recommendations_train.csv', 'filename': 'Hotel_Recommendations_train.csv', 'delimiter': ',', 'file_type': 'Delimeted', 'is_header': 'Use Header Line'}")
	#sourcePostExecutionHook(hotelrecommendationstrain)

except Exception as ex: 
	logging.error(ex)


***PERFORMING OPERATIONS***

In [ ]:
#%run aggregationrecommenderHooks.ipynb
try:
	aggregation = AggregationOperation.run(["6006a6ed76b1471523534d34"],{"6006a6ed76b1471523534d34": hotelrecommendationstrain}, "6006a6f176b1471523534d35", spark, "{'dataType': 'real', 'dataTypes': ['String', 'real', 'numeric'], 'url': '/FileStore/platform/demo/Hotel_Recommendations_train.csv', 'SelectedColumns': [{'columnName': 'shopping_mall', 'type': 'real', 'disable': False}, {'columnName': 'convenience_store', 'type': 'real', 'disable': False}], 'functionData': {'functionName': 'aggregate', 'applyOn': [{'columnName': 'shopping_mall', 'type': 'real', 'disable': False}, {'columnName': 'convenience_store', 'type': 'real', 'disable': False}], 'functionsData': [{'aggregateFunction': 'sum', 'aggregateOn': [{'columnName': 'shopping_mall', 'type': 'real', 'disable': False}, {'columnName': 'convenience_store', 'type': 'real', 'disable': False}], 'groupOn': [{'columnName': 'shopping_mall', 'type': 'real', 'disable': False}, {'columnName': 'convenience_store', 'type': 'real', 'disable': False}]}]}, 'allColumns': [{'columnName': 'user_id', 'type': 'String', 'disable': True}, {'columnName': 'hotel_name', 'type': 'String', 'disable': True}, {'columnName': 'rating', 'type': 'real', 'disable': False}, {'columnName': 'price', 'type': 'numeric', 'disable': True}, {'columnName': 'near_station', 'type': 'real', 'disable': False}, {'columnName': 'restaurant', 'type': 'real', 'disable': False}, {'columnName': 'entertain', 'type': 'real', 'disable': False}, {'columnName': 'shopping_mall', 'type': 'real', 'disable': False}, {'columnName': 'convenience_store', 'type': 'real', 'disable': False}, {'columnName': 'trip_type', 'type': 'String', 'disable': True}, {'columnName': 'companion', 'type': 'String', 'disable': True}, {'columnName': 'destination', 'type': 'String', 'disable': True}]}")

except Exception as ex: 
	logging.error(ex)


***TRANSFORMING DATAFRAME***

In [ ]:
#%run aggregationrecommenderHooks.ipynb
try:
	#transformationPreExecutionHook()

	autofe = TransformationMain.run(aggregation,json.dumps( {"FE": [{"transformationsData": [{"feature_label": "user_id", "transformation_label": "novalue"}], "feature": "user_id", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "84", "mean": "", "stddev": "", "min": "user1", "max": "user9", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "user_id"}, {"transformationsData": [{"feature_label": "hotel_name", "transformation_label": "novalue"}], "feature": "hotel_name", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "84", "mean": "", "stddev": "", "min": "128 Bangkok", "max": "Yoo Yen Pen Sook", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "hotel_name"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "rating", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "84", "mean": "8.67", "stddev": "0.48", "min": "7.0", "max": "9.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "rating"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "price", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "84", "mean": "2740.27", "stddev": "2168.08", "min": "280", "max": "9534", "missing": "0"}, "updatedLabel": "price"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "near_station", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "84", "mean": "0.19", "stddev": "0.09", "min": "0.0", "max": "0.390243902", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "near_station"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "restaurant", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "84", "mean": "0.36", "stddev": "0.14", "min": "0.2", "max": "0.8", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "restaurant"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "entertain", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "84", "mean": "0.12", "stddev": "0.08", "min": "0.0", "max": "0.289855072", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "entertain"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "shopping_mall", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "84", "mean": "0.13", "stddev": "0.08", "min": "0.0", "max": "0.350877193", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "shopping_mall"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "convenience_store", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "84", "mean": "0.19", "stddev": "0.08", "min": "0.0", "max": "0.6", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "convenience_store"}, {"transformationsData": [{"feature_label": "trip_type", "transformation_label": "novalue"}], "feature": "trip_type", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "84", "mean": "", "stddev": "", "min": "business", "max": "leisure", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "trip_type"}, {"transformationsData": [{"feature_label": "companion", "transformation_label": "novalue"}], "feature": "companion", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "84", "mean": "", "stddev": "", "min": "couple", "max": "solo", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "companion"}, {"transformationsData": [{"feature_label": "destination", "transformation_label": "novalue"}], "feature": "destination", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "84", "mean": "", "stddev": "", "min": "bangkok", "max": "bangkok", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "destination"}]}))

	#transformationPostExecutionHook(autofe)

except Exception as ex: 
	logging.error(ex)


***TRAIN MODEL***

In [ ]:
#%run aggregationrecommenderHooks.ipynb
try:
	#mlPreExecutionHook()

	recommendationsautoml = Recommender.run(autofe,spark,json.dumps( {"model": "LightFm", "autorecommend": 1, "rating": "rating", "user": "user_id", "product": "hotel_name", "originalfile": "/FileStore/platform/demo/Hotel_Recommendations_train.csv"}))

	#mlPostExecutionHook(recommendationsautoml)

except Exception as ex: 
	logging.error(ex)
